In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython import display
sns.set_theme(palette='colorblind')

Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [ ]:
# load the bike stations and yelp poi dataframes
bike_stations = pd.read_csv('../data/bike_stations_barcelona.csv', index_col=0)
# yelp_poi = pd.read_csv('../data/bike_stations_barcelona_poi_yelp.csv', index_col=0)

# Rename bike_stations id column to match yelp_poi so they can be merged
bike_stations = bike_stations.rename(columns={'id': 'station_id'})

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

Look at the data before and after the join to validate your data.